# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import urllib.parse
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [2]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 582


In [3]:
"""
    From docs: https://openweathermap.org/current
    https://api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}
"""

'\n    From docs: https://openweathermap.org/current\n    https://api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}\n'

---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [8]:
# Set the API base URL
url = 'https://api.openweathermap.org/data/2.5/weather'

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    QUERY_DICT = {
        'q': city,
        'appid': weather_api_key
    }

    query_string = urllib.parse.urlencode(QUERY_DICT)
    city_url = f'{url}?{query_string}' 
    print(city_url)
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        # city_date = city_weather

        # Append the City information into city_data list
        city_data.append({"City": city,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,})
                          # "Date": city_date})

        print(city_weather)
    except:
        print(f"couldnt find {city}")
    finally:
        print('sleeping for a second')
        time.sleep(1)
        
# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
https://api.openweathermap.org/data/2.5/weather?q=bethel&appid=a6d33a53a92cdb76f3d1eb83ac0c2b2b
Processing Record 1 of Set 1 | bethel
{'coord': {'lon': -73.414, 'lat': 41.3712}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 263.75, 'feels_like': 260.11, 'temp_min': 262.92, 'temp_max': 264.94, 'pressure': 1026, 'humidity': 60, 'sea_level': 1026, 'grnd_level': 1004}, 'visibility': 10000, 'wind': {'speed': 1.79, 'deg': 285, 'gust': 3.58}, 'clouds': {'all': 29}, 'dt': 1739949421, 'sys': {'type': 2, 'id': 2093006, 'country': 'US', 'sunrise': 1739965380, 'sunset': 1740004335}, 'timezone': -18000, 'id': 5282297, 'name': 'Bethel', 'cod': 200}
sleeping for a second
https://api.openweathermap.org/data/2.5/weather?q=puerto+natales&appid=a6d33a53a92cdb76f3d1eb83ac0c2b2b
Processing Record 2 of Set 1 | puerto natales
{'coord': {'lon': -72.4875, 'lat': -51.7

KeyboardInterrupt: 

In [9]:
city_data

[{'City': 'bethel',
  'Lat': 41.3712,
  'Lng': -73.414,
  'Max Temp': 264.94,
  'Humidity': 60,
  'Cloudiness': 29,
  'Wind Speed': 1.79,
  'Country': 'US'},
 {'City': 'puerto natales',
  'Lat': -51.7236,
  'Lng': -72.4875,
  'Max Temp': 282.63,
  'Humidity': 97,
  'Cloudiness': 77,
  'Wind Speed': 2.55,
  'Country': 'CL'},
 {'City': 'port-aux-francais',
  'Lat': -49.35,
  'Lng': 70.2167,
  'Max Temp': 278.78,
  'Humidity': 77,
  'Cloudiness': 47,
  'Wind Speed': 6.9,
  'Country': 'TF'},
 {'City': 'colchani',
  'Lat': -20.3,
  'Lng': -66.9333,
  'Max Temp': 284.75,
  'Humidity': 69,
  'Cloudiness': 100,
  'Wind Speed': 3.96,
  'Country': 'BO'},
 {'City': 'floro',
  'Lat': 61.5996,
  'Lng': 5.0328,
  'Max Temp': 273.13,
  'Humidity': 59,
  'Cloudiness': 40,
  'Wind Speed': 7.72,
  'Country': 'NO'},
 {'City': 'fale old settlement',
  'Lat': -9.3852,
  'Lng': -171.2468,
  'Max Temp': 301.8,
  'Humidity': 66,
  'Cloudiness': 91,
  'Wind Speed': 10.49,
  'Country': 'TK'}]

In [ ]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(

# Show Record Count
city_data_df.count()

In [ ]:
# Display sample data
city_data_df.head()

In [ ]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

In [ ]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [ ]:
# Build scatter plot for latitude vs. temperature
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig1.png")

# Show plot
plt.show()

#### Latitude Vs. Humidity

In [ ]:
# Build the scatter plots for latitude vs. humidity
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig2.png")

# Show plot
plt.show()

#### Latitude Vs. Cloudiness

In [ ]:
# Build the scatter plots for latitude vs. cloudiness
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig3.png")

# Show plot
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# Build the scatter plots for latitude vs. wind speed
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [ ]:
# Define a function to create Linear Regression plots
# YOUR CODE HERE

In [ ]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
# YOUR CODE HERE

# Display sample data
northern_hemi_df.head()

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
# YOUR CODE HERE

# Display sample data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
# Linear regression on Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Linear regression on Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE